In [ ]:
# import win10toast using pip install win10toast
import requests
import xml.etree.ElementTree as ET
import time
from win10toast import ToastNotifier

RSS_FEED_URL = "http://feeds.bbci.co.uk/news/rss.xml"

def loadRSS():
    '''Utility function to load RSS feed'''
    resp = requests.get(RSS_FEED_URL)
    return resp.content

def parseXML(rss):
    '''Utility function to parse XML format RSS feed'''
    root = ET.fromstring(rss)
    newsitems = []
    for item in root.findall('./channel/item'):
        news = {}
        for child in item:
            if child.tag == '{http://search.yahoo.com/mrss/}content':
                news['media'] = child.attrib['url']
            else:
                news[child.tag] = child.text
        newsitems.append(news)
    return newsitems

def topStories():
    '''Main function to generate and return news items'''
    rss = loadRSS()
    newsitems = parseXML(rss)
    return newsitems

# Fetch news items
newsitems = topStories()

# Initialize the Windows notification system (ToastNotifier)
toaster = ToastNotifier()

# Loop through the news items and display notifications
for newsitem in newsitems:
    title = newsitem.get('title', 'No Title')  # No need to decode
    description = newsitem.get('description', 'No Description')  # No need to decode
    
    # Show notification on screen
    toaster.show_toast(title, description, duration=2)

    # Wait for the notification to complete
    while toaster.notification_active():
        time.sleep(1)
